### 1. 重点回顾

数据库作用、分类

关系型数据库概念

	字段、记录、表 table、数据库 database

MySQL 服务操作

	ps aux | grep mysql
	
	sudo service mysql status/start/stop/restart

数据库登录和退出

linux>mysql -uroot -pxxx

数据类型和约束

1主键约束     面向系统 而不是给用户使用(包含唯一约束)

	唯一确定一行记录            
	
	MySQL 建议所有表的主键字段都叫 id, 类型为 int unsigned.
	
	学号

2非空约束   必填项  该字段不能**为空** NULL   紧急联系人

3唯一约束   该字段不允许**重复**                          手机号、身份证号

4默认约束   该字段不填值 就使用**默认值**        年龄

5外键约束   

		两个表  一个表提供数据<技能 主表>   另一个使用数据<英雄 子表>
	
		kongfuid 是外键<数据来自另一个表>
	
		要求外键字段kongfuid的取值必须在  主表中对应字段能够找到；否则失败
## 2. MySQL数据库

>    SQL 语句

### 2.1 数据库操作⭐️⭐️

>   SQL 语句默认以;结尾  不能出现全角的字符 
>
>   ctrl c 中断该 SQL 的编写

```sql
-- 1 展示系统中所有的数据库
show databases;

-- 2 切换到指定数据库使用
use py27;

-- 3 查看当前正在使用的数据库
select database();

-- 4 创建数据库  一定要指定字符集 否则可能    不能存中文
create database py28 charset=utf8;

-- 5 展示创建数据库的 SQL 语句
show create database py28;
CREATE DATABASE `py28` /*!40100 DEFAULT CHARACTER SET utf8 */

-- 如果创建数据库没有指定字符集默认为拉丁 数据库中就不能包含中文
❌create database python28;
CREATE DATABASE `python28` /*!40100 DEFAULT CHARACTER SET latin1
```

### 2.2 表结构操作⭐️⭐️

>字段名字  类型 约束

```sql
-- 1 展示当前数据库中所有的表
show tables;

-- 2 创建表  create table 表名(字段名 类型 约束,....);
-- auto_increment 自动递增
create table students(
id int unsigned primary key auto_increment COMMENT "学号",
name varchar(32) not null COMMENT "姓名",
age  tinyint unsigned default 18 COMMENT "年龄",
height decimal(5,2) COMMENT "身高",
gender enum('男','女','保密') COMMENT "性别");

-- 3 展示创建数据表的 SQL 语句
show create table students;

-- 4 查看表结构 describe 描述
desc students;

-- 5 修改表结构 添加、修改、删除
alter table 表名 
-- 5.1 添加字段
alter table 表名 add 字段名 类型 约束; 
alter table students add birthday datetime not null;
-- 5.2 修改字段的类型 约束
alter table 表名 modify 字段名 新类型 约束
alter table students modify birthday date not null;
-- 5.3 修改字段的名字 类型
alter table 表名 change 原字段名 新字段名 新类型 约束;
alter table students change birthday birth date not null;
-- 5.4 删除字段
alter table 表名 drop 字段名;
alter table students drop birth;
 
-- 6 删除表
drop table 表名;
drop table students;
```

### 2.3 表数据操作⭐️⭐️⭐️

```sql
-- 1 增  插入  insert into 表名(字段名1,.....) values (数据1,....),(数据1,.....)
-- 表名后面的字段列表和数据中字段   一一对应
-- 1.1 全列插入  插入数据中字段顺序和数量  和表中完全一样
insert into students(id,name,age,height,gender) values(0,'张三',28,167.66,'男');
insert into students(id,name,age,height,gender) values(0,'张三丰',108,177.66,'男'),(0,'郭襄',105,160.66,'女');
insert into students(id,name,age,height,gender) values(100,'张无忌',18,167.66,'男'),(null,'赵敏',18,167.66,'女');
-- 字段名在全列插入时 可以省略
insert into students values(null,'周芷若',18,165.66,'女');

-- 1.2 部分列插入  插入数据中字段顺序和数量  和表中不完全一样
insert into students(name,height,gender) values ('陈友谅',180.88,'男'),('东方不败',176,'保密');

-- 2 查  select 字段名1,... from 表名; 
-- * 表示所有字段
select name,age from students;
select * from students;

-- 3 删  delete from 表名 where 条件;
-- 3.1 物理删除 很难恢复
delete from students where id = 105;
-- 3.2 逻辑删除 本质就是给需要逻辑删除的数据做标记
--- 添加一个字段用以逻辑删除做标记
alter table students add is_delete bit default 0;
--- 逻辑删除 把标记改为 1
update students set is_delete = 1 where 条件;
update students set is_delete = 1 where id=106;
--- 撤销逻辑删除 把标记改为 0
update students set is_delete = 0 where 条件;

-- 4 改  update 表名 set 字段名=值[,字段2=值,..] where 条件
update students set name='岳不群',age=50 where id =106;
update students set name='岳不群' where id =106;
```

>   where 条件用以筛选记录的
>
>   mysql> select * from students where id<100;

### 2.4 as 关键字⭐️⭐️

****

作用: 给结果集中/当前 SQL 字段名/表名  **起别名**

形式:名字 as 别名/名字 别名

```sql
mysql> select id as '学号',name as '姓名',age as '高寿' from students;

-- 后面必须要给表起别名  比如  自连接
select students.id,students.name,students.age from students;
-- 一旦给字段和表起了别名 那么在当前 SQL 中不能再使用原名
--❌select students.id,students.name,students.age from students as s;

mysql> select s.id,s.name,s.age from students as s;

```

>   别名只在当前 SQL 有效，并且不影响表结构
>
>   as 在 MYSQL 中可以省略
>
>   select s.id,s.name,s.age from students s;

### 2.5 distinct关键字⭐️⭐️

作用: 对显示结果中重复行**去重**

形式: select distinct 字段  from 表;

````sql
select age,gender from students;
select distinct age,gender from students;
select gender from students;
select distinct gender from students;
````

![image-20190827110605404](day09笔记.assets/image-20190827110605404.png)

### 2.6 where子句⭐️⭐️⭐️

作用: 筛选表中记录

格式: select 字段 from 表名 where 字段 运算符 数据;

```sql
mysql> select * from students where id != 100;
mysql> select * from students where id <> 100;

select * from students where age >=20 and age <=150;
select * from students where age =18 or age=28 or age=108;

-- 2 like 模糊查询<知道部分线索>
-- % 通配任意个任意字符 类似* 
-- _ 通配一个任意字符  类似?
mysql> select * from students where name like '张%';
mysql> select * from students where name like '张三%';
select * from students where name like '张_';
mysql> select * from students where name like '张__';
mysql> select * from students where name like '张__%';
select * from students where name like '%三%';

-- 3 范围查询
-- 1 连续范围 
select * from students where age >=20 and age <=150;
select * from students where age between 20 and 150;
-- 2 非连续范围  字段 in 集合
select * from students where age =18 or age=28 or age=108;
select * from students where age in (18,28,108);

-- 3 判空操作 IS NULL是空; 是非空 IS NOT NULL   isn't
insert into students (name,gender) values('程咬金','男');
--❌select * from stundents where height = NULL;
select * from students where height IS NULL;
--❌select * from students where height NOT IS NULL;
select * from students where height IS NOT NULL;
```



### 2.7 排序⭐️⭐️⭐️

作用: 按照数据的某个多个属性进行从大到小、从小到大的排列

​	比如 百度搜索引擎  按照网页匹配程度、点击量、竞价、优先级、紧急程度

按多个字段排序:  如果前面字段值相等 就使用后面字段排序

形式:  order by 字段名1 方式1,字段2 方式2,....

>   方式 默认是升序 ASC  ascend  向上倾斜  ; DESC decend 向下倾斜

```sql
select * from students order by height;
select * from students order by height ASC;
select * from students order by height DESC;

-- 先按年龄 相等按身高  相等按 id 排
mysql> select * from students order by age,height,id;
-- 身高降序, 年龄升序, id降序
select * from students order by age,height DESC;
select * from students order by height DESC,age,id DESC;
```

>   把前面字段值相等的记录  继续使用后续字段进行排序  

### 2.8 分页⭐️⭐️⭐️

作用: 服务器数据太多，不能一次性全部给用户，将所有数据分成一页一页的，在用户需要的使用提供下一页。

格式: limit [数据起始下标=0,] 数据数量

```sql
select * from students order by height DESC limit 3;
select * from students order by height DESC limit 0,3;

-- 取身高最高的 第 4 名到第 6 名(下标3,4,5)
select * from students order by height DESC limit 3,3;

-- 推导公式  
--第 1 页 取 5 个数据  limit 0,5
--第 2 页 取 5 个数据  limit 5,5
--第 3 页 取 5 个数据  limit 10,5
--第 n 页 取 m 个数据  limit m*(n-1),m
```

### 2.9 聚合函数⭐️⭐️⭐️

作用:  将一组数据运算得到一个值， 也称组函数

格式:  聚合函数(字段名)

>   所有聚合都不统计字段为 NULL 的记录

```sql
-- 1 求平均 求出班级中平均身高  avg()
select avg(height) from students;

-- 2 求数量 求出班级中的人数   count()
select count(height) from students;
select count(*) from students;

-- 3 求和   求出班级中所有人的年龄总和  sum()
select sum(age) from students;
mysql> select sum(height)/count(height) from students;

-- 4 最大值 max
select max(height) from students;
-- 5 最小值 min
select min(age) from students;
```



>   聚合函数相关的特殊函数
>
>   mysql> select sum(ifnull(height,200)) from students;
>
>   枚举    gender enum ('男','女','保密')
>
>   select * from students where gender = '男';
>
>   在判断枚举类型值的时候 还可以使用元素对应的编号(从 1 开始,2,3,...)

### 2.10 分组⭐️⭐️⭐️

作用: 按照一个多个字段划分，字段相同的就在同一个组，可以针对每个组进行分别的操作。

格式: group by 字段名

```sql
❌select * from students group by gender;
select gender from students group by gender;
-- 一旦分组  结果集的结构已经发生变化<三维结构>不能直接使用SQL 查看每个字段  需要通过group_concat/聚合函数

-- group_concat作用就是把分组中每个成员的某个字段拼接在一起显示
-- 1 性别分组  查看每个分组中每个成员的信息(名字)
mysql> select gender,group_concat(name) from students group by gender;

-- 2 分组加聚合函数select count(*) from students;
-- 默认情况如果没有分组 聚合函数会把整个表当做一个组进行统计
-- 一旦按某字段分组了 聚合函数就把每个组分别统计
select gender,count(*) from students group by gender;
select gender,avg(age) from students group by gender;

mysql> select address,avg(height) from students group by address;

-- 3 分组筛选条件  
-- where 筛选表中记录的   having 筛选分组中结果的
mysql> select address,avg(height) from students group by address    having avg(height) > 170;
mysql> select address,count(*)  from students group by address having count(*) > 3;
select address,count(*) as c from students group by address having c > 3;

-- 4 最后一行新增汇总行
select address,avg(height) from students group by address;
| 上海    |  171.330000 |
| 北京    |  168.660000 |
| 深圳    |  174.270000 |
select address,avg(height) from students group by address with rollup;
| 上海    |  171.330000 |
| 北京    |  168.660000 |
| 深圳    |  174.270000 |
| NULL   |  171.093333 |
```

### 2.11 连接⭐️⭐️⭐️

作用: 当做结果集中字段数据来自多个表  ，需要将多个表进行连接操作

格式: 表1 连接符  表2 连接条件 连接符2  表3 连接条件2

```mysql
-- 准备数据用以连接代码演示    
create table hero(id int unsigned primary key auto_increment,name varchar(64) not null,gongfuid int unsigned not null
);
create table gongfu(id int unsigned primary key auto_increment,name varchar(64) not null
);
insert into hero (name,gongfuid) values ('妲己',2),('李白',1),('程咬金',3),('公孙离',5);
insert into gongfu (name) values ('吟诗作对'),('魅惑'),('三板斧'),('沉默');
```

```sql
-- 1 笛卡尔积 左表每一行和右表每一行分别拼接在一起 
-- select * from 左表 join 右表;
select * from hero join gongfu;

/* 1 | 妲己      |        2 |  1 | 吟诗作对     |
|  2 | 李白      |        1 |  1 | 吟诗作对     |
|  3 | 程咬金    |        3 |  1 | 吟诗作对     |
|  4 | 公孙离    |        5 |  1 | 吟诗作对     |
|  1 | 妲己      |        2 |  2 | 魅惑         |
|  2 | 李白      |        1 |  2 | 魅惑         |
|  3 | 程咬金    |        3 |  2 | 魅惑         |
|  4 | 公孙离    |        5 |  2 | 魅惑         |
|  1 | 妲己      |        2 |  3 | 三板斧       |
|  2 | 李白      |        1 |  3 | 三板斧       |
|  3 | 程咬金    |        3 |  3 | 三板斧       |
|  4 | 公孙离    |        5 |  3 | 三板斧       |
|  1 | 妲己      |        2 |  4 | 沉默         |
|  2 | 李白      |        1 |  4 | 沉默         |
|  3 | 程咬金    |        3 |  4 | 沉默         |
|  4 | 公孙离    |        5 |  4 | 沉默         */
-- 2 内连接  inner 可以省略
-- on 是用来筛选笛卡尔积中满足条件的数据
-- 左表 inner inner join 右表 on 左表.gongfuid = 右表.id
select * from hero inner join gongfu on hero.gongfuid = gongfu.id;

/*  2 | 李白      |        1 |  1 | 吟诗作对     |
|  1 | 妲己      |        2 |  2 | 魅惑         |
|  3 | 程咬金    |        3 |  3 | 三板斧       */

-- 3 左外连接 outer 可以省略
-- 外部数据来自左表 右表位置以 NULL 填充
-- 左表 left outer join 右表 on 左表.gongfuid = 右表.id
mysql> select * from hero left outer join gongfu on hero.gongfuid = gongfu.id;
/* 2 | 李白      |        1 |    1 | 吟诗作对     |
|  1 | 妲己      |        2 |    2 | 魅惑         |
|  3 | 程咬金    |        3 |    3 | 三板斧       |
|  4 | 公孙离    |        5 | NULL | NULL        */

-- 4 右外 right outer join /right join
-- 外部数据来自右表 左表位置以 NULL 填充
-- 左表 right outer join 右表 on 左表.gongfuid = 右表.id
mysql> select * from hero right outer join gongfu on hero.gongfuid = gongfu.id;
/*   1 | 妲己      |        2 |  2 | 魅惑         |
|    2 | 李白      |        1 |  1 | 吟诗作对     |
|    3 | 程咬金    |        3 |  3 | 三板斧       |
| NULL | NULL      |     NULL |  4 | 沉默        */
```





### 2.12 SQL执行顺序⭐️

>   主要了解 关键字书写顺序、运行顺序()

```sql
(8)SELECT (9)DISTINCT<select_list>
(1)FROM <left_table>
(2)<join_type>JOIN<right_table>
(3)ON <join_condition>
(4)WHERE <where_condition>
(5)GROUP BY <group_by_list>
(6)WITH {CUBE|ROLLUP}
(7)HAVING <having_condition>
(10)ORDER BY<order_by_list>
(11)LIMIT <limit_number>
```





### 2.13  自连接⭐️⭐️⭐️

左表  inner join 右表  on 左表.字段 符号 右表.字段;

>   左表和右表都是一个表、

需求分析: 

​	城市表 inner join 省份表 on 城市表.上级机构编号 等于 省份表.当前机构编号;

城市表和省份表来自同一个表

>   左表和右表不能是  同一个名字 需要给左表和右表起别名

>   选择收货地址   进行地址联动

```sql
create table areas(
    id varchar(30) not null primary key, 
    title varchar(30), 
    pid varchar(30)
);
```

```sql
-- areas city inner join areas pro on city.pid = pro.id  看作一个表  select * from 表;
select * from areas city inner join areas pro on city.pid = pro.id ;

-- 求出广东省下属所有的市
mysql> select * from areas city inner join areas pro on city.pid = pro.id  where pro.title = '广东省';

select * from areas city inner join areas pro on city.pid = pro.id  where pro.title = '河南省';
```



### 2.14 子查询 ⭐️⭐️⭐️

​	一个语句中嵌套了另一个小的select语句<子查询>

分类: 标量、行、列

```sql
-- 1 需求: 求出班级中身高 高于平均身高 的学生信息
-- 标量子查询   子查询返回的结果是一个值
-- 1.1 先求出班级中平均身高
select avg(height) from students;
-- 1.2 根据平均身高求出结果
mysql> select * from students where height > 171.09;
mysql> select * from students where height > ( select avg(height) from students);

-- 2 需求: 求出班级中 身高最高 并且年龄最大的同学
-- 行子查询
-- 2.1 先求出最高的身高 最大的年龄
select max(height),max(age) from students;

-- 2.2 根据上一步结果求出  身高=最高 年龄=最大的同学信息
select * from students where height=187 and age = 108;
select * from students where (height,age)=(187,108);
select * from students where (height,age)=
	(select max(height),max(age) from students);

-- 3 需求: 求出所有正在被英雄使用的技能名称
-- 3.1 先求出所有英雄使用技能 id  返回一列多行数据
select gongfuid from hero;
-- 3.2 根据上一步结果 
select name from gongfu where id in (2,1,3,5);
mysql> select name from gongfu where id in (select gongfuid from hero);

-- 连接 当连接和子查询都可以解决问题时 推荐使用连接
mysql> select gongfu.name from hero 
	inner join gongfu on hero.gongfuid = gongfu.id;
```



### 2.15 外键⭐️⭐️

作用:  子表使用来自主表的数据，  子表对应数据字段定义为外键foreign key

​	这个字段的设置/修改将会受到外键字段的约束(外键字段的取值需要在主表对应字段存在)

```sql
-- 查看  hero_ibfk_1就是外键约束名字
show create table hero;
 CREATE TABLE `hero` (
  `id` int(10) unsigned NOT NULL AUTO_INCREMENT,
  `name` varchar(64) NOT NULL,
  `gongfuid` int(10) unsigned NOT NULL,
  PRIMARY KEY (`id`),
  KEY `gongfuid` (`gongfuid`),
  CONSTRAINT `hero_ibfk_1` FOREIGN KEY (`gongfuid`) REFERENCES `gongfu` (`id`)
) 

-- 添加 要求表中已有数据必须满足外键约束要求
alter table 表名 add foreign key(字段) references 主表(字段);
alter table hero add foreign key(gongfuid) references gongfu(id);

-- 删除  需要知道外键约束名字   
alter table hero drop foreign key hero_ibfk_1;
```



### 2.16 数据库编程⭐️⭐️⭐️

通过编程方式操作数据库。

使用 pymysql

#### 2.16.1 编程步骤

```python
import pymysql

# 1 创建连接
连接对象=pymysql.connect(host,port,
                     user,password,database,charset)

# 2 获取游标
游标对象 = 连接对象.cursor()

# 3 使用 游标执行 SQL 语句
影响行数 = 游标对象.execute(sql,[SQL所需参数,...])

# 4 获取结果/提交修改
游标对象.fetchall()  .fetchone()  / 连接对象.commit()

# 5 关闭游标 连接
游标对象.close()  连接对象.close()
```



#### 2.16.2 查询数据⭐️⭐️⭐️

```python
import pymysql

if __name__ == '__main__':
    # 1 建立连接            服务器IP,端口,用户名,密码,数据库名,通信采用的字符集
    conn = pymysql.connect(host='127.0.0.1',port=3306,user='root',password='mysql',
                    database='py28',charset='utf8')
    # 2 通过连接对象获取游标对象
    cur = conn.cursor()

    # 3 通过游标执行execute SQL 语句   execute参数是需要执行的ＳＱＬ语句　返回值是ＳＱＬ影响的行数
    sql = "select * from hero;"
    row_count = cur.execute(sql)
    print("影响的行数有%s" % row_count)

    # 4 通过游标取出数据
    # 4.1 一次性获取全部数据　((一行数据),...) 取完返回()
    # print(cur.fetchall())
    # print(cur.fetchall())

    # 4.1 一行一行数据的获取　(一行数据)　　取完返回None
    print(cur.fetchone()[1])

    # 5 先关闭游标 关闭连接
    cur.close()
    conn.close()

```



#### 2.16.3 增删改⭐️⭐️⭐️

```python
import pymysql

if __name__ == '__main__':
    # 1 建立连接
    conn = pymysql.connect(host='localhost', port=3306, user='root', password='mysql',
                    db='py28', charset='utf8')
    # 2 创建游标
    cur = conn.cursor()

    # 3 使用游标执行ＳＱＬ
    # sql = "update hero set gongfuid = 3000 where id = 4";
    # sql = "insert into hero(name,gongfuid) VALUES ('墨子', 200),('韩信',100)"
    sql = "delete from hero where id = 2;"
    row_count = cur.execute(sql)   # 执行ＳＱＬ语句
    print("ＳＱＬ影响的行数%s" % row_count)

    # 4 获取结果/提交修改　　保存到数据库
    conn.commit()

    # 5 关闭连接
    cur.close()
    conn.close()
```

#### 2.16.4 SQL注入问题⭐️⭐️

原因: 后台将功能和用户输入的数据进行**字符串拼接**
	    用户输入带有恶意的数据 导致整个 SQL 语义发生变化

解决方案:  不能拼接 应该使用参数化列表

​	SQL 所需参数放到一个列表中  当做参数给 execute 第二个参数

>   注意事项  参数化列表的情况下占位符只能是%s 左右不需要引号

```python
import pymysql

if __name__ == '__main__':
    # 1 建立连接
    conn = pymysql.connect(host='localhost', port=3306, user='root', password='mysql',
                    db='py28', charset='utf8')
    # 2 创建游标
    cur = conn.cursor()

    # 3 使用游标执行ＳＱＬ
    # sql = "update hero set gongfuid = 3000 where id = 4";
    # sql = "insert into hero(name,gongfuid) VALUES ('墨子', 200),('韩信',100)"
    # sql = "delete from hero where id = 2;"
    id = input("请输入你需要查看的英雄id：")
    name = input("请输入你需要查看的英雄名字：")

    # sql = "select * from hero where id='%s' and name='%s'" % (id,name)
    # sql = "select * from hero where id='1' and name='妲己'"
    # sql = "select * from hero where id='1'＃ and name='妲己'"
    # sql = "select * from hero where id='1' or 1=1 ＃ and name='妲己'"
    sql = "select * from hero where id=%s and name=%s"  #　参数化列表的情况下占位符只能是%s 左右不需要'

    row_count = cur.execute(sql, (id,name))   # 执行ＳＱＬ语句
    # select * from hero where id='1\' or 1=1 #' and name='fajdlfkdas'
    print(cur.mogrify(sql, [id,name])) # 处理之后的SQL
    print("ＳＱＬ影响的行数%s" % row_count)

    # 4 获取结果/提交修改　　保存到数据库
    # conn.commit()
    print(cur.fetchall())
    # 5 关闭连接
    cur.close()
    conn.close()
```



### 2.17 数据库设计⭐️

>   设计好需要几张表 、每个表有几个字段

#### 2.17.1 设计范式

1NF  要求字段不可再分-原子性

2NF  1NF;要求表中有主键; 要求非主键字段必须完全依赖主键

```
2NF 要求非主键字段完全依赖于主键 而不能部分依赖

主键: 订单 ID    产品 ID
非主键字段  产品单价、名字只和产品 ID有关系，而
	产品 ID 只是主键的一部分 是为部分依赖
```

3NF 2NF; 要求非主键字段必须直接依赖于主键

```
3NF 要求非主键字段直接依赖于主键 而不能传递依赖
主键是 订单 ID  非主键是用户 ID/名称、地址
其中非主键  名称、地址只依赖于用户ID   而用户 ID不是主键 是为传递依赖  
```

>   关于数据库设计 没有最好的规则 只要适应业务的设计
>
>   反范式化设计

#### 2.17.2 ER 图

​	椭圆 属性    对应数据库中的  字段<外键关联>

​	矩形 实体    对应数据库中的  表 table

​	直线 关联

​	菱形  关系    对应数据库中  分情况

​	1:1 1:多          对应数据库中   字段<外键关联>

​	多对多          对应数据库中    中间表table(无法直接表示 需要转化中间关系表）

### 2.18 事务⭐️⭐️

概念: SQL语句的序列，要么全部执行成功，要么什么都不执行。

应用场景:   订单

特性: ACID

​	原子性: 事务是一个整体，不可拆分

​	一致性: 数据库总是从一个一致性状态切换到另一个一致性状态

​		40000      200000   一致性

​		20000      200000   不一致性

​		20000      220000   一致性

​	隔离性:  事务与外界之间 中间状态相互不可见

​	持久性:   数据一旦保存到数据 数据库将会永久保存

>   一旦提交  不可撤销

提交 commit 将修改保存到数据中存储

回滚 rollback 将修改丢弃



命令行默认  自动开启事务<updaet delete insert>自动提交

命令行    begin;start transaction;                        commit;       rollback;

pymysql      自动开启事务                      连接对象.commit()     rollback()

>   可以关闭命令行自动提交     set autocommit =0;  需要手动提交

>    表数据操作updaet delete insert可以回滚   除此之外大多数不可以被回滚 
>
>   不是所有操作都可以被回滚   那些不能回滚的操作   称为隐式提交
>   常见如  修改表结构  删表结构  删数据  创建数据库



----------

明天讲    

索引

SQL 演练后面的代码 

京东案例<01-学习资料中>

# mysql批量导入数据:

```mysql
    1.素材/areas.sql
    2.复制到 meiduo_mall/scripts/areas.sql
    终端 cd meiduo_mall/scripts/
    执行 下面导入数据
    mysql -h数据库ip地址 -u数据库用户名 -p数据库密码 数据库 < areas.sql
    mysql -h127.0.0.1 -uroot -pmysql meiduo < areas.sql
    
 ```